In [2]:
import os
import pandas as pd

# read the text file and add the column names
read_file = pd.read_csv(r"./booksummries.txt", sep='	', header=None)
read_file.columns = ['ID', 'm number', 'book name', 'author name', 'date', 'label', 'summary']

# clean data
read_file['label'] = read_file['label'].str.replace(r'/m/\S*\s', '')
read_file['label'] = read_file['label'].str.replace(r'{', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'\\u00e0\s+clef', '')

# select columns
new_file = read_file.loc[:, ['book name', 'label', 'summary']]

#delete the columns with no labels
new_file.dropna(axis = 0, how = 'any', inplace = True)
new_file = new_file.iloc[:, [0, 2, 1]]

#output data as csv
new_file.to_csv(r'./booksummries.csv', index=False)

In [ ]:
def cosine_similarity(a, b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [ ]:
def calculation(label_list):
    for i in range(len(label_list)):
        j = i+1
        if (j < len(label_list)):
            for j in range(len(label_list)):
                a = nlp(label_list[i]).vector
                b = nlp(label_list[j]).vector
                if cosine_similarity(a, b)>0.6:
                        if(len(label_list[i])>len(label_list[j])):
                            label_list[i] = label_list[j]
                        else:
                            label_list[j] = label_list[i]
    return set(label_list)

In [ ]:
read_file["label"] = read_file["label"].fillna('')

In [ ]:

for index in range(len(read_file['label'])):
    label = read_file['label'][index].replace('"', ''). lower()
    label_list = re.split(', ', label)  
    label_list = calculation(label_list)
    read_file.xs(index)['label']= label_list

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def words_process(new_file):
    book_summaries = new_file['summary']
    summary_list = [summary for summary in book_summaries]
    summary_num = len(summary_list)
    #summaries = ''.join(summary_list)
    print("the total number of books: {}\n".format(summary_num))
    
    all_docs = []

    for doc in summary_list:
        # Tokenize the string into words
        tokens = word_tokenize(doc)
        # Remove non-alphabetic tokens, such as punctuation
        words = [word.lower() for word in tokens if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if not word in stop_words]
        all_docs.append(words)
        
    return all_docs

In [ ]:
words_process(new_file.iloc[1:3, :])

In [ ]:
from gensim.models import KeyedVectors

# Load word2vec model (trained on an enormous Google corpus)
google_vecs = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)
vec_dim = google_vecs.vector_size

def words_embedding(words):
    

    # Filter the list of vectors to include only those that Word2Vec has a vector for
    vector_list = [google_vecs[word] for word in words if word in google_vecs.vocab]

    # Create a list of the words corresponding to these vectors
    words_filtered = [word for word in words if word in google_vecs.vocab]

    #Zip the words together with their vector representations
    word_vec_zip = zip(words_filtered, vector_list)

    # Cast to a dict so we can turn it into a DataFrame
    word_vec_dict = dict(word_vec_zip)
    word_vec = pd.DataFrame.from_dict(word_vec_dict, orient='index')
    
    return word_vec


In [ ]:
words_embedding(all_docs[1])

In [ ]:
import numpy as np
def avg_doc_embedding(all_docs):
    doc_vec = []
    for doc in all_docs:
        if len(doc) != 0:
            word_vec =  word_embedding(doc)
            doc_vec.append(np.mean(np.array(word_vec), axis=1))
    
    summary_vec = np.array(doc_vec)
    return doc_vec


In [ ]:
avg_doc_embedding(all_docs)